In [11]:
import os

def maybe_mkdir_p(directory: str) -> None:
    os.makedirs(directory, exist_ok=True)

maic_dir = '/mnt/backup/'
base_dir = os.path.join(maic_dir, 'working')
input_dir = '/mnt/dataset/'
temp_dir = '/mnt/temp/'

maybe_mkdir_p(base_dir)
maybe_mkdir_p(temp_dir)

# ! git clone https://github.com/keemsir/nnUNet.git

respository_dir = os.path.join(maic_dir, 'nnUNet')
os.chdir(respository_dir)

! pip install -e .

os.chdir(base_dir)

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
Obtaining file:///mnt/backup/nnUNet
  Attempting uninstall: nnunet
    Found existing installation: nnunet 1.7.0
    Can't uninstall 'nnunet'. No files were found to uninstall.
  Running setup.py develop for nnunet
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [12]:
# installed
import os
import shutil
import json
import numpy as np

from collections import OrderedDict
from scipy import special
import copy
import cv2
from tqdm import tqdm
import h5py

# must install
import pydicom
import nibabel as nib
import SimpleITK as sitk

In [13]:
task_dir

NameError: name 'task_dir' is not defined

In [21]:
task_name = 'Task44_PETCT'
convert_name = 'Task444_PETCT'

main_dir = os.path.join(base_dir, 'nnUNet/nnunet')
mainT_dir = os.path.join(temp_dir, 'nnUNet/nnunet')

rawbase_dir = os.path.join(mainT_dir, 'nnUNet_raw_data_base/')

pp_dir = os.path.join(mainT_dir, 'preprocessed')
tasks_dir = os.path.join(mainT_dir, 'Tasks')
task_dir = os.path.join(tasks_dir, task_name)

model_dir = os.path.join(main_dir, 'nnUNet_trained_models')
Prediction_dir = os.path.join(main_dir, 'nnUNet_Prediction_Results')
result_dir = os.path.join(Prediction_dir, convert_name)

staple_dir = os.path.join(Prediction_dir, 'staple')

# 1. Data preprocessing
maybe_mkdir_p(tasks_dir)
maybe_mkdir_p(temp_dir)

# 2. Directory
maybe_mkdir_p(main_dir)
maybe_mkdir_p(model_dir)
maybe_mkdir_p(pp_dir)

# 3. Directory
maybe_mkdir_p(result_dir)
maybe_mkdir_p(staple_dir)


#Environment Setting
os.environ['nnUNet_raw_data_base'] = rawbase_dir #os.path.join(mainT_dir, 'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = pp_dir #os.path.join(mainT_dir, 'preprocessed')
os.environ['RESULTS_FOLDER'] = model_dir #os.path.join(main_dir, 'nnUNet_trained_models')


print('Setting Completed!')

Setting Completed!


In [22]:
def hdf2nifti(hdf_folder: str, save_folder: str):
    # hdf_folder : [train_dir, test_dir] hdf5 file path
    # save_folder : [imagesTr, imagesTs] Save Folder path
    maybe_mkdir_p(os.path.join(save_folder, 'imagesTr'))
    maybe_mkdir_p(os.path.join(save_folder, 'labelsTr'))
    maybe_mkdir_p(os.path.join(save_folder, 'imagesTs'))
    maybe_mkdir_p(os.path.join(save_folder, 'labelsTs'))
    print('Creating "{}" Image & Label ..'.format(os.path.basename(os.path.normpath(save_folder))))
    hdf5_files = os.listdir(hdf_folder)


    for hdf5_file in hdf5_files[:60]:


        hdf5_path = os.path.join(hdf_folder, hdf5_file)

        # image
        f_i = h5py.File(hdf5_path, 'r')
        ctarr = np.asarray(f_i['CT'])
        petarr = np.asarray(f_i['PET'])
        labels = np.asarray(f_i['Aorta'])
        f_i.close()

        SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT = ctarr.shape
        images = np.empty([SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT, 0], dtype=np.single)

        image_ct = np.expand_dims(ctarr, axis=3)
        images = np.append(images, image_ct, axis=3)
        image_pet = np.expand_dims(petarr, axis=3)
        images = np.append(images, image_pet, axis=3)


        hdf5_file_NAME = hdf5_file

        niim = nib.Nifti1Image(images, affine=np.eye(4))
        nib.save(niim, os.path.join(save_folder, 'imagesTr/{}.nii.gz'.format(hdf5_file[:-8])))

        nila = nib.Nifti1Image(labels, affine=np.eye(4))
        nib.save(nila, os.path.join(save_folder, 'labelsTr/{}.nii.gz'.format(hdf5_file[:-8])))


    print('"{}" Image & Label Completed !!'.format(os.path.basename(os.path.normpath(save_folder))))
    print('Image Patient : {}'.format(len(os.listdir(input_dir))))
    
    for hdf5_file in hdf5_files[60:]:


        hdf5_path = os.path.join(hdf_folder, hdf5_file)

        # image
        f_i = h5py.File(hdf5_path, 'r')
        ctarr = np.asarray(f_i['CT'])
        petarr = np.asarray(f_i['PET'])
        labels = np.asarray(f_i['Aorta'])
        f_i.close()

        SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT = ctarr.shape
        images = np.empty([SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT, 0], dtype=np.single)

        image_ct = np.expand_dims(ctarr, axis=3)
        images = np.append(images, image_ct, axis=3)
        image_pet = np.expand_dims(petarr, axis=3)
        images = np.append(images, image_pet, axis=3)


        hdf5_file_NAME = hdf5_file

        niim = nib.Nifti1Image(images, affine=np.eye(4))
        nib.save(niim, os.path.join(save_folder, 'imagesTs/{}.nii.gz'.format(hdf5_file[:-8])))

        nila = nib.Nifti1Image(labels, affine=np.eye(4))
        nib.save(nila, os.path.join(save_folder, 'labelsTs/{}.nii.gz'.format(hdf5_file[:-8])))


    print('"{}" Image & Label Completed !!'.format(os.path.basename(os.path.normpath(save_folder))))
    print('Image Patient : {}'.format(len(os.listdir(input_dir))))

hdf2nifti(input_dir, task_dir)

Creating "Task44_PETCT" Image & Label ..
"Task44_PETCT" Image & Label Completed !!
Image Patient : 80
"Task44_PETCT" Image & Label Completed !!
Image Patient : 80


In [34]:
def json_mk(save_dir: str):
    # Path
    imagesTr = os.path.join(save_dir, 'imagesTr')
    imagesTs = os.path.join(save_dir, 'imagesTs')
    maybe_mkdir_p(imagesTr)
    maybe_mkdir_p(imagesTs)

    overwrite_json_file = True
    json_file_exist = False

    if os.path.exists(os.path.join(save_dir, 'dataset.json')):
        print('dataset.json already exist!')
        json_file_exist = True

    if json_file_exist == False or overwrite_json_file:

        json_dict = OrderedDict()
        json_dict['name'] = "Kidney"
        json_dict['description'] = "Medical Image AI Challenge 2021"
        json_dict['tensorImageSize'] = "3D"
        json_dict['reference'] = "https://maic.or.kr/competitions/"
        json_dict['licence'] = "SNUH"
        json_dict['release'] = "18/10/2021"

        json_dict['modality'] = {
            "0": "CT",
            "1": "PET"
        }
        json_dict['labels'] = {
            "0": "background",
            "1": "Aorta"
        }

        train_ids = sorted(os.listdir(imagesTr))
        test_ids = sorted(os.listdir(imagesTs))
        json_dict['numTraining'] = len(train_ids)
        json_dict['numTest'] = len(test_ids)

        json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

        json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids] #(i[:i.find("_0000")])

        with open(os.path.join(save_dir, "dataset.json"), 'w') as f:
            json.dump(json_dict, f, indent=4, sort_keys=False)

        if os.path.exists(os.path.join(save_dir, 'dataset.json')):
            if json_file_exist == False:
                print('dataset.json created!')
            else:
                print('dataset.json overwritten!')
json_mk(task_dir)

dataset.json created!


In [36]:
os.listdir('/mnt/temp/nnUNet/nnunet/Tasks/Task44_PETCT')

['imagesTr', 'labelsTr', 'imagesTs', 'labelsTs', 'dataset.json']

In [37]:
os.chdir(mainT_dir)
!nnUNet_convert_decathlon_task -i /mnt/temp/nnUNet/nnunet/Tasks/Task44_PETCT -output_task_id 444 # -i : task_dir
!nnUNet_plan_and_preprocess -t 444 # --verify_dataset_integrity
os.chdir(base_dir)



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

23090557_20130
23090560_20160
23090562_20140
23090564_20130
23090568_20121
23090571_20120
23090578_20120
23090581_20130
before crop: (2, 299, 128, 128) after crop: (2, 299, 128, 128) spacing: [1. 1. 1.] 

before crop: (2, 299, 128, 128) after crop: (2, 299, 128, 128) spacing:

In [ ]:
# Model 1
os.chdir(main_dir)

!nnUNet_train 2d nnUNetTrainerV2 444 0
!nnUNet_train 2d nnUNetTrainerV2 444 1
!nnUNet_train 2d nnUNetTrainerV2 444 2
!nnUNet_train 2d nnUNetTrainerV2 444 3
!nnUNet_train 2d nnUNetTrainerV2 444 4

os.chdir(base_dir)